# 欢迎来到香山 Tutorial - GEM5 部分

## 1. 编译运行 GEM5

In [ ]:
%%bash

pushd ../ && source env.sh && popd

check_env() {
    if [[ -z "${gem5_home}" ]]; then
        echo "Error: No gem5_home found" 1>&2
        return 1
    fi
}

prepare_gem5() {
    if [ -f "$gem5_home/ext/dramsim3/DRAMsim3/libdramsim3.so" ]; then
        echo "DRAMSim3 already built, skip building DRAMSim3"
        return 0
    fi
    pushd $gem5_home && \
    cd ext/dramsim3 && \
    (stat DRAMsim3 || git clone https://github.com/umd-memsys/DRAMSim3.git DRAMsim3) && \
    cd DRAMsim3 && mkdir -p build && cd build && cmake .. && make -j `nproc` && \
    popd
}

build_gem5() {
    if [ -f "$gem5_home/build/RISCV/gem5.opt" ]; then
        echo "gem5 already built, skip building gem5"
        return 0
    fi
    pushd $gem5_home && \
    scons build/RISCV/gem5.opt --linker=mold -j `nproc` && \
    popd
}

build_nemu_diff() {
    if [ -f "$gem5_home/riscv64-nemu-interpreter-c1469286ca32-so" ]; then
        echo "NEMU diff already exist, skip downloading"
        return 0
    fi
    # Used for difftest and GCPT restorer
    pushd $gem5_home && \
    wget https://github.com/OpenXiangShan/GEM5/releases/download/2024-10-16/riscv64-nemu-interpreter-c1469286ca32-so && \
    popd
}

prepare_data_proc() {
    # Validated commit for tutorial: 4000c092b8bde21fd4aa493f9907fa100dbcb3fc
    pushd xs-gem5 && \
    (stat gem5_data_proc || git clone https://github.com/shinezyy/gem5_data_proc.git) && \
    pushd gem5_data_proc && \
    git reset --hard 4000c092b8bde21fd4aa493f9907fa100dbcb3fc && \
    pip3 install -r requirements.txt && \
    popd && \
    popd
}

check_env && prepare_gem5 && build_gem5 && build_nemu_diff && prepare_data_proc

## 2. 运行 CoreMark

In [ ]:
%%bash
#!/usr/bin/env bash

pushd ../ && source env.sh && popd

pushd $gem5_home && \
export LD_LIBRARY_PATH=$gem5_home/ext/dramsim3/DRAMsim3:$LD_LIBRARY_PATH && \
export GCBV_REF_SO=$gem5_home/riscv64-nemu-interpreter-c1469286ca32-so && \
mkdir -p util/xs_scripts/coremark && \
cd util/xs_scripts/coremark && \
$gem5_home/build/RISCV/gem5.opt $gem5_home/configs/example/xiangshan.py \
--raw-cpt --generic-rv-cpt=$NOOP_HOME/ready-to-run/coremark-2-iteration.bin && \
popd

## 3. 查看 TopDown 计数器

In [ ]:
%%bash
#!/usr/bin/env bash

pushd ../ >/dev/null && source env.sh >/dev/null && popd >/dev/null

pushd xs-gem5/gem5_data_proc && \
python3 batch.py \
  -s $gem5_home/util/xs_scripts/coremark \
  -t --topdown-raw && \
popd

## 4. 查看 Cache MPKI数据

在这里，我们采用的是一个已经运行完成的 SPEC06 Simpoint 测试结果。你可以在 `data` 目录下找到相关的文件。

In [ ]:
%%bash
#!/usr/bin/env bash

pushd ../ >/dev/null && source env.sh >/dev/null && popd >/dev/null

pushd xs-gem5/gem5_data_proc && \
mkdir -p results && \
export PYTHONPATH=`pwd` && \
ulimit -n 65535 && \
python3 batch.py -s ../data/xs-model-l1bank -o gem5-cache-example.csv --cache > /dev/null && \
python3 simpoint_cpt/compute_weighted.py \
    -r gem5-cache-example.csv \
    -j ../data/xs-model-l1bank/cluster-0-0.json \
    -o weighted.csv | tail -n 33 && \
popd


## 5. SPEC06 算分

在这里，我们采用的是一个已经运行完成的 SPEC06 Simpoint 测试结果。你可以在 `data` 目录下找到相关的文件。

In [ ]:
%%bash

#!/usr/bin/env bash

pushd ../ >/dev/null && source env.sh >/dev/null && popd >/dev/null

pushd xs-gem5/gem5_data_proc && \
mkdir -p results && \
export PYTHONPATH=`pwd` && \
ulimit -n 65535 && \
python3 batch.py -s ../data/xs-model-l1bank -o gem5-score-example.csv > /dev/null && \
python3 simpoint_cpt/compute_weighted.py \
    -r gem5-score-example.csv \
    -j ../data/xs-model-l1bank/cluster-0-0.json \
    --score score.csv | tail -n 41 && \
popd


## 6. GEM5 添加 Prefetcher

底下是一个很大的脚本，我们重点关心 Shell 函数 `self_build_despacito_stream`

```bash
function self_build_despacito_stream() {
    if [ -f "$gem5_home/../tutorial/xs-gem5/data/gem5.opt.despacito_stream" ]; then
        return
    fi
    pushd $gem5_home && \
    cd .. && git submodule update --init gem5 && cd gem5 && \
    (git branch -D add_a_new_prefetcher >/dev/null 2>&1 || true) && \
    (git checkout -b add_a_new_prefetcher || true) && \
    git am -3 ../tutorial/xs-gem5/DespacitoStream.patch && \
    scons build/RISCV/gem5.opt --linker=mold -j `nproc` && \
    cp build/RISCV/gem5.opt ../tutorial/xs-gem5/data/gem5.opt.despacito_stream && \
    popd
}
```

在这里，我们创建了一个新的分支 `add_a_new_prefetcher`，并应用了一个 patch `DespacitoStream.patch`。这个 patch 为 GEM5 添加了一个新的预取器 `DespacitoStream`。在应用完 patch 后，我们编译了 GEM5，并将生成的可执行文件保存到 `data/gem5.opt.despacito_stream`。

在进行了一些其他整理操作后，我们运行了 mcf 切片，并计算得到最终的数据。

```bash
function diff_result() {
    pushd $gem5_home > /dev/null && \
    echo "===== Results =====" && \
    echo -n "Baseline                IPC: " && \
    cat util/xs_scripts/mcf_baseline/m5out/stats.txt | grep "system.cpu.ipc" | tr -s ' ' | cut -d ' ' -f2 && \
    echo -n "Despactio Stream        IPC: " && \
    cat util/xs_scripts/mcf_despacito_stream/m5out/stats.txt | grep "system.cpu.ipc" | tr -s ' ' | cut -d ' ' -f2 && \
    echo -n "Baseline         L1D Misses: " && \
    cat util/xs_scripts/mcf_baseline/m5out/stats.txt | grep "system.cpu.dcache.ReadReq.misses::total" | tr -s ' ' | cut -d ' ' -f2 && \
    echo -n "Despactio Stream L1D Misses: " && \
    cat util/xs_scripts/mcf_despacito_stream/m5out/stats.txt | grep "system.cpu.dcache.ReadReq.misses::total" | tr -s ' ' | cut -d ' ' -f2 && \
    popd > /dev/null
}
```

In [ ]:
%%bash

#!/usr/bin/env bash

pushd ../ >/dev/null && source env.sh >/dev/null && popd >/dev/null

function run_baseline() {
    gem5_opt_path=$1
    pushd $gem5_home && \
    export LD_LIBRARY_PATH=$gem5_home/ext/dramsim3/DRAMsim3:$LD_LIBRARY_PATH && \
    export GCBV_REF_SO=$gem5_home/riscv64-nemu-interpreter-c1469286ca32-so && \
    mkdir -p util/xs_scripts/mcf_baseline && \
    cd util/xs_scripts/mcf_baseline && \
    $gem5_opt_path $gem5_home/configs/example/xiangshan.py \
    --generic-rv-cpt=$gem5_home/../tutorial/xs-gem5/data/mcf_12253_0.137576_.zstd \
    --gcpt-restorer=$gem5_home/../tutorial/xs-gem5/data/normal-gcb-restorer.bin \
    -I 300000 && \
    popd
}

function run_with_despacito_stream() {
    gem5_opt_path=$1
    pushd $gem5_home && \
    export LD_LIBRARY_PATH=$gem5_home/ext/dramsim3/DRAMsim3:$LD_LIBRARY_PATH && \
    export GCBV_REF_SO=$gem5_home/riscv64-nemu-interpreter-c1469286ca32-so && \
    mkdir -p util/xs_scripts/mcf_despacito_stream && \
    cd util/xs_scripts/mcf_despacito_stream && \
    $gem5_opt_path $gem5_home/configs/example/xiangshan.py \
    --generic-rv-cpt=$gem5_home/../tutorial/xs-gem5/data/mcf_12253_0.137576_.zstd \
    --gcpt-restorer=$gem5_home/../tutorial/xs-gem5/data/normal-gcb-restorer.bin \
    -I 300000 && \
    popd
}

function diff_result() {
    pushd $gem5_home > /dev/null && \
    echo "===== Results =====" && \
    echo -n "Baseline                IPC: " && \
    cat util/xs_scripts/mcf_baseline/m5out/stats.txt | grep "system.cpu.ipc" | tr -s ' ' | cut -d ' ' -f2 && \
    echo -n "Despactio Stream        IPC: " && \
    cat util/xs_scripts/mcf_despacito_stream/m5out/stats.txt | grep "system.cpu.ipc" | tr -s ' ' | cut -d ' ' -f2 && \
    echo -n "Baseline         L1D Misses: " && \
    cat util/xs_scripts/mcf_baseline/m5out/stats.txt | grep "system.cpu.dcache.ReadReq.misses::total" | tr -s ' ' | cut -d ' ' -f2 && \
    echo -n "Despactio Stream L1D Misses: " && \
    cat util/xs_scripts/mcf_despacito_stream/m5out/stats.txt | grep "system.cpu.dcache.ReadReq.misses::total" | tr -s ' ' | cut -d ' ' -f2 && \
    popd > /dev/null
}

function self_build_baseline() {
    if [ -f "$gem5_home/../tutorial/xs-gem5/data/gem5.opt" ]; then
        return
    fi
    pushd $gem5_home && \
    cd .. && git submodule update --init gem5 && cd gem5 && \
    scons build/RISCV/gem5.opt --linker=mold -j `nproc` && \
    cp build/RISCV/gem5.opt ../tutorial/xs-gem5/data/gem5.opt && \
    popd
}

function self_build_despacito_stream() {
    if [ -f "$gem5_home/../tutorial/xs-gem5/data/gem5.opt.despacito_stream" ]; then
        return
    fi
    pushd $gem5_home && \
    cd .. && git submodule update --init gem5 && cd gem5 && \
    (git branch -D add_a_new_prefetcher >/dev/null 2>&1 || true) && \
    (git checkout -b add_a_new_prefetcher || true) && \
    git am -3 ../tutorial/xs-gem5/DespacitoStream.patch && \
    scons build/RISCV/gem5.opt --linker=mold -j `nproc` && \
    cp build/RISCV/gem5.opt ../tutorial/xs-gem5/data/gem5.opt.despacito_stream && \
    popd
}

self_build_baseline && self_build_despacito_stream && \
run_baseline `realpath xs-gem5/data/gem5.opt` && \
run_with_despacito_stream `realpath xs-gem5/data/gem5.opt.despacito_stream` && \
diff_result
